# Predictive Accuracy

## Packages

In [2]:
import matplotlib.pyplot as plt
import os

import pandas as pd
import numpy as np
import math
import torch
from matplotlib import pyplot as plt
import time

# GPBoost
import gpboost as gpb

# For CRPS & Log-Score
from scipy.stats import norm
from scipy.special import erf

In [6]:
toy = True

### Generate Data

In [7]:
#import numpy as np
#import gpytorch
#from gpytorch.means import ConstantMean
#from gpytorch.kernels import ScaleKernel, MaternKernel, InducingPointKernel
#from gpytorch.distributions import MultivariateNormal
#from gpytorch.models import ApproximateGP
#from gpytorch.variational import CholeskyVariationalDistribution
#from gpytorch.variational import VariationalStrategy
# Number of input dimensions
#vector_d = [2,10]
## number of reps
#num_reps = 10
#for i, val1 in enumerate(vector_d):

#    # Ranges
#    ranges = torch.tensor([(0.15, 0.3,0.45,0.6,0.75)])
#    avg_dist_5 = np.sqrt(5/6)
#    if val1 != 5:
#        # average distance
#        avg_dist = np.sqrt(val1/6)
#        min_ratio = avg_dist_5/0.15
#        max_ratio = avg_dist_5/0.75
#        min_range = avg_dist/min_ratio
#        max_range = avg_dist/max_ratio
#        ranges = torch.linspace(min_range, max_range, val1)
#        ranges = ranges.unsqueeze(0)
        
#    ranges = 2.448/3 * ranges # Matern05
#    ranges = 2.448/2.74 * ranges # Matern15
#    ranges = 2.448/2.647 * ranges
#    ranges = ranges # Gaussian
#    print(ranges)
#    for j in range(num_reps):
#        print(j)
#        X, y, _ = simulate_gp_response("matern",40000, 0.3, 1.0, ranges,j,1.5)
#        data = pd.DataFrame(X.numpy(), columns=[f"x{i+1}" for i in range(X.shape[1])])
#        data['y'] = y.numpy()

### Functions

In [8]:
# CRPS
def crps_norm_vectorized(observations, means, sigmas):
    """
    Compute CRPS for multiple normal distributions.
    
    Parameters:
    observations (array-like): Observed values
    means (array-like): Means of the forecast distributions
    sigmas (array-like): Variance of the forecast distributions
    
    Returns:
    array-like: CRPS values
    """
    observations = np.asarray(observations)
    means = np.asarray(means)
    sigmas = np.sqrt(np.asarray(sigmas))
    sigmas = np.where((sigmas == 0) | np.isnan(sigmas), 1e-06, sigmas)
    z = (observations - means) / sigmas
    crps = sigmas * (z * (2 * norm.cdf(z) - 1) + 2 * norm.pdf(z) - 1 / np.sqrt(np.pi))
    return np.mean(crps)

# Log-Score
def log_score_norm_vectorized(observations, means, sigmas):
    """
    Compute Log Score for multiple normal distributions.
    
    Parameters:
    observations (array-like): Observed values
    means (array-like): Means of the forecast distributions
    sigmas (array-like): Variance of the forecast distributions
    
    Returns:
    array-like: Log Score values
    """
    observations = np.asarray(observations)
    means = np.asarray(means)
    sigmas = np.sqrt(np.asarray(sigmas))
    sigmas = np.where((sigmas == 0) | np.isnan(sigmas), 1e-06, sigmas)
    return np.mean(-norm.logpdf(observations, means, sigmas))

### Matern05

In [11]:
# number of inducing points
num_ind_points = 200
# Different number of Vecchia neighbors
num_vecchia_neighbors = 30
# Number of input dimensions
vector_d = [2,10]
# number of reps
num_reps = 10
if toy:
    num_reps = 2
# Define row and column names
row_names = ['2-d','10-d']
col_names = ['Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6',
             'Iteration 7', 'Iteration 8', 'Iteration 9', 'Iteration 10']

# List of matrix names
matrix_names = [
    "matrix_t_vecchia", "matrix_t_fitc", "matrix_t_fsva_corr", "matrix_t_fsva_eucl",
    "matrix_tp_vecchia", "matrix_tp_fitc", "matrix_tp_fsva_corr", "matrix_tp_fsva_eucl",
    "matrix_RMSE_vecchia", "matrix_RMSE_fitc", "matrix_RMSE_fsva_corr", "matrix_RMSE_fsva_eucl",
    "matrix_LS_vecchia", "matrix_LS_fitc", "matrix_LS_fsva_corr", "matrix_LS_fsva_eucl",
    "matrix_CRPS_vecchia", "matrix_CRPS_fitc", "matrix_CRPS_fsva_corr", "matrix_CRPS_fsva_eucl",
    "matrix_RMSE_exp_vecchia", "matrix_RMSE_exp_fitc", "matrix_RMSE_exp_fsva_corr", "matrix_RMSE_exp_fsva_eucl",
    "matrix_LS_exp_vecchia", "matrix_LS_exp_fitc", "matrix_LS_exp_fsva_corr", "matrix_LS_exp_fsva_eucl",
    "matrix_CRPS_exp_vecchia", "matrix_CRPS_exp_fitc", "matrix_CRPS_exp_fsva_corr", "matrix_CRPS_exp_fsva_eucl"
]

# Create empty matrices and assign them dynamically to the global scope
for matrix_name in matrix_names:
    globals()[matrix_name] = pd.DataFrame(np.zeros((len(row_names), len(col_names))), 
                      index=row_names, columns=col_names)

# Nested loop to iterate over both vectors
for i, val1 in enumerate(vector_d):
    # Ranges
    ranges = torch.tensor([(0.15, 0.3,0.45,0.6,0.75)])
    avg_dist_5 = np.sqrt(5/6)
    if val1 != 5:
        # average distance
        avg_dist = np.sqrt(val1/6)
        min_ratio = avg_dist_5/0.15
        max_ratio = avg_dist_5/0.75
        min_range = avg_dist/min_ratio
        max_range = avg_dist/max_ratio
        ranges = torch.linspace(min_range, max_range, val1)
        ranges = ranges.unsqueeze(0)
    ranges = 2.448/3 * ranges
    print(ranges)
    flattened = np.asarray(ranges).reshape(-1)
    result = np.concatenate(([0.3, 1], flattened))
    for j in range(num_reps):
        print(j)
        url = f"https://raw.githubusercontent.com/TimGyger/VIF/refs/heads/main/Simulation/Data/Matern05/simulated_data_Matern05_{val1}_{j}.txt"

        # Load the CSV file directly from the URL
        df = pd.read_csv(url)

        # Select the first d columns for X
        X = df.iloc[:, :val1]  # First d columns

        # Select the last column for y
        y = df.iloc[:, -1]  # Last column
        
        # Construct Test and Train sets 
        corner = 1 - 1/(4**(1/val1))
        X = torch.tensor(X.values, dtype=torch.float32)
        y = torch.tensor(y.values, dtype=torch.float32)
        upper_corner_mask = torch.all(X > corner, dim = 1)
        
        X_exp_test = X[upper_corner_mask]
        y_exp_test =  y[upper_corner_mask]
        X_remaining = X[~upper_corner_mask]
        y_remaining = y[~upper_corner_mask]
        X_int_test = X_remaining[:10000]
        y_int_test = y_remaining[:10000]
        X_train = X_remaining[10000:20000]
        y_train = y_remaining[10000:20000]
        
        print(X_train.size())
        print(X_exp_test.size())
        print(X_int_test.size())
        
        
        X_train = np.asarray(X_train)
        y_train = np.asarray(y_train)
        X_int_test = np.asarray(X_int_test)
        y_int_test = np.asarray(y_int_test)
        X_exp_test = np.asarray(X_exp_test)
        y_exp_test = np.asarray(y_exp_test)
        
        start_time = time.time()
        # Vecchia
        model_vecchia = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 0.5,
                                 likelihood="gaussian",num_neighbors = num_vecchia_neighbors,
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 10)
        model_vecchia.set_optim_params(params={"init_cov_pars": result})
        model_vecchia.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_vecchia.loc[row_names[i], col_names[j]] = end_time - start_time

        start_time = time.time()
        pred_vecchia = model_vecchia.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_vecchia.loc[row_names[i], col_names[j]] = end_time - start_time
        matrix_RMSE_vecchia.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_vecchia['mu'])))
        matrix_CRPS_vecchia.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_vecchia['mu'], pred_vecchia['var'])
        matrix_LS_vecchia.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_vecchia['mu'], pred_vecchia['var'])

        pred_vecchia_exp = model_vecchia.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_vecchia.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_vecchia_exp['mu'])))
        matrix_CRPS_exp_vecchia.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_vecchia_exp['mu'], pred_vecchia_exp['var'])
        matrix_LS_exp_vecchia.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_vecchia_exp['mu'], pred_vecchia_exp['var'])

        start_time = time.time()
        # FITC
        model_FITC = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 0.5,
                                 likelihood="gaussian",num_ind_points = 200,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="fitc",seed = 10)
        
        model_FITC.set_optim_params(params={"init_cov_pars": result})
        
        model_FITC.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fitc.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fitc = model_FITC.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fitc.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fitc.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fitc['mu'])))
        matrix_CRPS_fitc.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fitc['mu'], pred_fitc['var'])
        matrix_LS_fitc.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fitc['mu'], pred_fitc['var'])

        pred_fitc_exp = model_FITC.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fitc.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fitc_exp['mu'])))
        matrix_CRPS_exp_fitc.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fitc_exp['mu'], pred_fitc_exp['var'])
        matrix_LS_exp_fitc.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fitc_exp['mu'], pred_fitc_exp['var'])

        start_time = time.time()
        # FSVA_eucl
        model_FSVA_eucl = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 0.5,num_neighbors = num_vecchia_neighbors,
                                 likelihood="gaussian",num_ind_points = num_ind_points,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 2)
        model_FSVA_eucl.set_optim_params(params={"init_cov_pars": result})
        model_FSVA_eucl.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fsva_eucl.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fsva_eucl = model_FSVA_eucl.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fsva_eucl.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fsva_eucl.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fsva_eucl['mu'])))
        matrix_CRPS_fsva_eucl.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fsva_eucl['mu'], pred_fsva_eucl['var'])
        matrix_LS_fsva_eucl.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fsva_eucl['mu'], pred_fsva_eucl['var'])

        pred_fsva_eucl_exp = model_FSVA_eucl.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fsva_eucl_exp['mu'])))
        matrix_CRPS_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fsva_eucl_exp['mu'], pred_fsva_eucl_exp['var'])
        matrix_LS_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fsva_eucl_exp['mu'], pred_fsva_eucl_exp['var'])

        
        
        start_time = time.time()
        # FSVA_corr
        model_FSVA_corr = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 0.5,num_neighbors = num_vecchia_neighbors,
                                 likelihood="gaussian",num_ind_points = num_ind_points,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 4)
        model_FSVA_corr.set_optim_params(params={"init_cov_pars": result})
        model_FSVA_corr.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fsva_corr.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fsva_corr = model_FSVA_corr.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fsva_corr.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fsva_corr.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fsva_corr['mu'])))
        matrix_CRPS_fsva_corr.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fsva_corr['mu'], pred_fsva_corr['var'])
        matrix_LS_fsva_corr.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fsva_corr['mu'], pred_fsva_corr['var'])

        pred_fsva_corr_exp = model_FSVA_corr.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fsva_corr.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fsva_corr_exp['mu'])))
        matrix_CRPS_exp_fsva_corr.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fsva_corr_exp['mu'], pred_fsva_corr_exp['var'])
        matrix_LS_exp_fsva_corr.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fsva_corr_exp['mu'], pred_fsva_corr_exp['var'])
        

tensor([[0.0774, 0.3871]])
0
torch.Size([10000, 2])
torch.Size([9916, 2])
torch.Size([10000, 2])
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing points have been determined 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing points have been determined 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing points have been determined 
1
torch.Size([10000, 2])
torch.Size([9967, 2])
torch.Size([10000, 2])
[GPBoost] [Warning] There is no intercept for modeling a possibly non-zero mean of the random effects. Consider including an intercept (= a column of 1's) in the covariates 'X' 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing points have been determined 
[GPBoost] [Warning] There is no intercept for modeling a possibly non-zero mean of the random effects. Consider including a

### Matern 1.5

In [26]:
# number of inducing points
num_ind_points = 200
# Different number of Vecchia neighbors
num_vecchia_neighbors = 30
# Number of input dimensions
vector_d = [2,10]
# number of reps
num_reps = 10
if toy:
    num_reps = 2
# Define row and column names
row_names = ['2-d','5-d','10-d','20-d','50-d','100-d']
col_names = ['Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6',
             'Iteration 7', 'Iteration 8', 'Iteration 9', 'Iteration 10']
if toy:
    row_names = ['2-d','10-d']
# List of matrix names
matrix_names = [
    "matrix_t_vecchia", "matrix_t_fitc", "matrix_t_fsva_corr", "matrix_t_fsva_eucl",
    "matrix_tp_vecchia", "matrix_tp_fitc", "matrix_tp_fsva_corr", "matrix_tp_fsva_eucl",
    "matrix_RMSE_vecchia", "matrix_RMSE_fitc", "matrix_RMSE_fsva_corr", "matrix_RMSE_fsva_eucl",
    "matrix_LS_vecchia", "matrix_LS_fitc", "matrix_LS_fsva_corr", "matrix_LS_fsva_eucl",
    "matrix_CRPS_vecchia", "matrix_CRPS_fitc", "matrix_CRPS_fsva_corr", "matrix_CRPS_fsva_eucl",
    "matrix_RMSE_exp_vecchia", "matrix_RMSE_exp_fitc", "matrix_RMSE_exp_fsva_corr", "matrix_RMSE_exp_fsva_eucl",
    "matrix_LS_exp_vecchia", "matrix_LS_exp_fitc", "matrix_LS_exp_fsva_corr", "matrix_LS_exp_fsva_eucl",
    "matrix_CRPS_exp_vecchia", "matrix_CRPS_exp_fitc", "matrix_CRPS_exp_fsva_corr", "matrix_CRPS_exp_fsva_eucl"
]

# Create empty matrices and assign them dynamically to the global scope
for matrix_name in matrix_names:
    globals()[matrix_name] = pd.DataFrame(np.zeros((len(row_names), len(col_names))), 
                      index=row_names, columns=col_names)

# Nested loop to iterate over both vectors
for i, val1 in enumerate(vector_d):
    # Ranges
    ranges = torch.tensor([(0.15, 0.3,0.45,0.6,0.75)])
    avg_dist_5 = np.sqrt(5/6)
    if val1 != 5:
        # average distance
        avg_dist = np.sqrt(val1/6)
        min_ratio = avg_dist_5/0.15
        max_ratio = avg_dist_5/0.75
        min_range = avg_dist/min_ratio
        max_range = avg_dist/max_ratio
        ranges = torch.linspace(min_range, max_range, val1)
        ranges = ranges.unsqueeze(0)
    ranges = 2.448/2.74 * ranges
    print(ranges)
    flattened = np.asarray(ranges).reshape(-1)
    result = np.concatenate(([0.3, 1], flattened))
    for j in range(num_reps):
        print(j)
        url = f"https://raw.githubusercontent.com/TimGyger/VIF/refs/heads/main/Simulation/Data/Matern15/simulated_data_Matern15_{val1}_{j}.txt"

        # Load the CSV file directly from the URL
        df = pd.read_csv(url)

        # Select the first d columns for X
        X = df.iloc[:, :val1]  # First d columns

        # Select the last column for y
        y = df.iloc[:, -1]  # Last column
        
        # Construct Test and Train sets 
        corner = 1 - 1/(4**(1/val1))
        X = torch.tensor(X.values, dtype=torch.float32)
        y = torch.tensor(y.values, dtype=torch.float32)
        upper_corner_mask = torch.all(X > corner, dim = 1)
        
        X_exp_test = X[upper_corner_mask]
        y_exp_test =  y[upper_corner_mask]
        X_remaining = X[~upper_corner_mask]
        y_remaining = y[~upper_corner_mask]
        X_int_test = X_remaining[:10000]
        y_int_test = y_remaining[:10000]
        X_train = X_remaining[10000:20000]
        y_train = y_remaining[10000:20000]
        
        print(X_train.size())
        print(X_exp_test.size())
        print(X_int_test.size())
        
        
        X_train = np.asarray(X_train)
        y_train = np.asarray(y_train)
        X_int_test = np.asarray(X_int_test)
        y_int_test = np.asarray(y_int_test)
        X_exp_test = np.asarray(X_exp_test)
        y_exp_test = np.asarray(y_exp_test)
        
        start_time = time.time()
        # Vecchia
        model_vecchia = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 1.5,
                                 likelihood="gaussian",num_neighbors = num_vecchia_neighbors,
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 10)
        model_vecchia.set_optim_params(params={"init_cov_pars": result})
        model_vecchia.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_vecchia.loc[row_names[i], col_names[j]] = end_time - start_time

        start_time = time.time()
        pred_vecchia = model_vecchia.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_vecchia.loc[row_names[i], col_names[j]] = end_time - start_time
        matrix_RMSE_vecchia.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_vecchia['mu'])))
        matrix_CRPS_vecchia.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_vecchia['mu'], pred_vecchia['var'])
        matrix_LS_vecchia.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_vecchia['mu'], pred_vecchia['var'])

        pred_vecchia_exp = model_vecchia.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_vecchia.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_vecchia_exp['mu'])))
        matrix_CRPS_exp_vecchia.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_vecchia_exp['mu'], pred_vecchia_exp['var'])
        matrix_LS_exp_vecchia.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_vecchia_exp['mu'], pred_vecchia_exp['var'])

        start_time = time.time()
        # FITC
        model_FITC = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 1.5,
                                 likelihood="gaussian",num_ind_points = 200,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="fitc",seed = 10)
        
        model_FITC.set_optim_params(params={"init_cov_pars": result})
        
        model_FITC.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fitc.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fitc = model_FITC.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fitc.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fitc.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fitc['mu'])))
        matrix_CRPS_fitc.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fitc['mu'], pred_fitc['var'])
        matrix_LS_fitc.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fitc['mu'], pred_fitc['var'])

        pred_fitc_exp = model_FITC.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fitc.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fitc_exp['mu'])))
        matrix_CRPS_exp_fitc.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fitc_exp['mu'], pred_fitc_exp['var'])
        matrix_LS_exp_fitc.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fitc_exp['mu'], pred_fitc_exp['var'])

        start_time = time.time()
        # FSVA_eucl
        model_FSVA_eucl = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 1.5,num_neighbors = num_vecchia_neighbors,
                                 likelihood="gaussian",num_ind_points = num_ind_points,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 2)
        model_FSVA_eucl.set_optim_params(params={"init_cov_pars": result})
        model_FSVA_eucl.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fsva_eucl.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fsva_eucl = model_FSVA_eucl.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fsva_eucl.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fsva_eucl.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fsva_eucl['mu'])))
        matrix_CRPS_fsva_eucl.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fsva_eucl['mu'], pred_fsva_eucl['var'])
        matrix_LS_fsva_eucl.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fsva_eucl['mu'], pred_fsva_eucl['var'])

        pred_fsva_eucl_exp = model_FSVA_eucl.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fsva_eucl_exp['mu'])))
        matrix_CRPS_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fsva_eucl_exp['mu'], pred_fsva_eucl_exp['var'])
        matrix_LS_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fsva_eucl_exp['mu'], pred_fsva_eucl_exp['var'])

        
        
        start_time = time.time()
        # FSVA_corr
        model_FSVA_corr = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 1.5,num_neighbors = num_vecchia_neighbors,
                                 likelihood="gaussian",num_ind_points = num_ind_points,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 4)
        model_FSVA_corr.set_optim_params(params={"init_cov_pars": result})
        model_FSVA_corr.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fsva_corr.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fsva_corr = model_FSVA_corr.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fsva_corr.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fsva_corr.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fsva_corr['mu'])))
        matrix_CRPS_fsva_corr.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fsva_corr['mu'], pred_fsva_corr['var'])
        matrix_LS_fsva_corr.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fsva_corr['mu'], pred_fsva_corr['var'])

        pred_fsva_corr_exp = model_FSVA_corr.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fsva_corr.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fsva_corr_exp['mu'])))
        matrix_CRPS_exp_fsva_corr.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fsva_corr_exp['mu'], pred_fsva_corr_exp['var'])
        matrix_LS_exp_fsva_corr.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fsva_corr_exp['mu'], pred_fsva_corr_exp['var'])
        

tensor([[0.0848, 0.4238]])
0
torch.Size([10000, 2])
torch.Size([9916, 2])
torch.Size([10000, 2])
[GPBoost] [Warning] There is no intercept for modeling a possibly non-zero mean of the random effects. Consider including an intercept (= a column of 1's) in the covariates 'X' 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing points have been determined 
[GPBoost] [Warning] There is no intercept for modeling a possibly non-zero mean of the random effects. Consider including an intercept (= a column of 1's) in the covariates 'X' 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing points have been determined 
[GPBoost] [Warning] There is no intercept for modeling a possibly non-zero mean of the random effects. Consider including an intercept (= a column of 1's) in the covariates 'X' 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing

### Matern 2.5

In [29]:
# number of inducing points
num_ind_points = 200
# Different number of Vecchia neighbors
num_vecchia_neighbors = 30
# Number of input dimensions
vector_d = [2,10]
# number of reps
num_reps = 10
if toy:
    num_reps = 2
# Define row and column names
row_names = ['2-d','5-d','10-d','20-d','50-d','100-d']
col_names = ['Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6',
             'Iteration 7', 'Iteration 8', 'Iteration 9', 'Iteration 10']
if toy:
    row_names = ['2-d','10-d']
# List of matrix names
matrix_names = [
    "matrix_t_vecchia", "matrix_t_fitc", "matrix_t_fsva_corr", "matrix_t_fsva_eucl",
    "matrix_tp_vecchia", "matrix_tp_fitc", "matrix_tp_fsva_corr", "matrix_tp_fsva_eucl",
    "matrix_RMSE_vecchia", "matrix_RMSE_fitc", "matrix_RMSE_fsva_corr", "matrix_RMSE_fsva_eucl",
    "matrix_LS_vecchia", "matrix_LS_fitc", "matrix_LS_fsva_corr", "matrix_LS_fsva_eucl",
    "matrix_CRPS_vecchia", "matrix_CRPS_fitc", "matrix_CRPS_fsva_corr", "matrix_CRPS_fsva_eucl",
    "matrix_RMSE_exp_vecchia", "matrix_RMSE_exp_fitc", "matrix_RMSE_exp_fsva_corr", "matrix_RMSE_exp_fsva_eucl",
    "matrix_LS_exp_vecchia", "matrix_LS_exp_fitc", "matrix_LS_exp_fsva_corr", "matrix_LS_exp_fsva_eucl",
    "matrix_CRPS_exp_vecchia", "matrix_CRPS_exp_fitc", "matrix_CRPS_exp_fsva_corr", "matrix_CRPS_exp_fsva_eucl"
]

# Create empty matrices and assign them dynamically to the global scope
for matrix_name in matrix_names:
    globals()[matrix_name] = pd.DataFrame(np.zeros((len(row_names), len(col_names))), 
                      index=row_names, columns=col_names)

# Nested loop to iterate over both vectors
for i, val1 in enumerate(vector_d):
    # Ranges
    ranges = torch.tensor([(0.15, 0.3,0.45,0.6,0.75)])
    avg_dist_5 = np.sqrt(5/6)
    if val1 != 5:
        # average distance
        avg_dist = np.sqrt(val1/6)
        min_ratio = avg_dist_5/0.15
        max_ratio = avg_dist_5/0.75
        min_range = avg_dist/min_ratio
        max_range = avg_dist/max_ratio
        ranges = torch.linspace(min_range, max_range, val1)
        ranges = ranges.unsqueeze(0)
    ranges = 2.448/2.647 * ranges
    print(ranges)
    flattened = np.asarray(ranges).reshape(-1)
    result = np.concatenate(([0.3, 1], flattened))
    for j in range(num_reps):
        print(j)
        url = f"https://raw.githubusercontent.com/TimGyger/VIF/refs/heads/main/Simulation/Data/Matern25/simulated_data_Matern25_{val1}_{j}.txt"

        # Load the CSV file directly from the URL
        df = pd.read_csv(url)

        # Select the first d columns for X
        X = df.iloc[:, :val1]  # First d columns

        # Select the last column for y
        y = df.iloc[:, -1]  # Last column
        
        # Construct Test and Train sets 
        corner = 1 - 1/(4**(1/val1))
        X = torch.tensor(X.values, dtype=torch.float32)
        y = torch.tensor(y.values, dtype=torch.float32)
        upper_corner_mask = torch.all(X > corner, dim = 1)
        
        X_exp_test = X[upper_corner_mask]
        y_exp_test =  y[upper_corner_mask]
        X_remaining = X[~upper_corner_mask]
        y_remaining = y[~upper_corner_mask]
        X_int_test = X_remaining[:10000]
        y_int_test = y_remaining[:10000]
        X_train = X_remaining[10000:20000]
        y_train = y_remaining[10000:20000]
        
        print(X_train.size())
        print(X_exp_test.size())
        print(X_int_test.size())
        
        
        X_train = np.asarray(X_train)
        y_train = np.asarray(y_train)
        X_int_test = np.asarray(X_int_test)
        y_int_test = np.asarray(y_int_test)
        X_exp_test = np.asarray(X_exp_test)
        y_exp_test = np.asarray(y_exp_test)
        
        start_time = time.time()
        # Vecchia
        model_vecchia = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 2.5,
                                 likelihood="gaussian",num_neighbors = num_vecchia_neighbors,
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 10)
        model_vecchia.set_optim_params(params={"init_cov_pars": result})
        model_vecchia.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_vecchia.loc[row_names[i], col_names[j]] = end_time - start_time

        start_time = time.time()
        pred_vecchia = model_vecchia.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_vecchia.loc[row_names[i], col_names[j]] = end_time - start_time
        matrix_RMSE_vecchia.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_vecchia['mu'])))
        matrix_CRPS_vecchia.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_vecchia['mu'], pred_vecchia['var'])
        matrix_LS_vecchia.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_vecchia['mu'], pred_vecchia['var'])

        pred_vecchia_exp = model_vecchia.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_vecchia.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_vecchia_exp['mu'])))
        matrix_CRPS_exp_vecchia.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_vecchia_exp['mu'], pred_vecchia_exp['var'])
        matrix_LS_exp_vecchia.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_vecchia_exp['mu'], pred_vecchia_exp['var'])

        start_time = time.time()
        # FITC
        model_FITC = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 2.5,
                                 likelihood="gaussian",num_ind_points = 200,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="fitc",seed = 10)
        
        model_FITC.set_optim_params(params={"init_cov_pars": result})
        
        model_FITC.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fitc.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fitc = model_FITC.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fitc.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fitc.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fitc['mu'])))
        matrix_CRPS_fitc.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fitc['mu'], pred_fitc['var'])
        matrix_LS_fitc.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fitc['mu'], pred_fitc['var'])

        pred_fitc_exp = model_FITC.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fitc.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fitc_exp['mu'])))
        matrix_CRPS_exp_fitc.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fitc_exp['mu'], pred_fitc_exp['var'])
        matrix_LS_exp_fitc.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fitc_exp['mu'], pred_fitc_exp['var'])

        start_time = time.time()
        # FSVA_eucl
        model_FSVA_eucl = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 2.5,num_neighbors = num_vecchia_neighbors,
                                 likelihood="gaussian",num_ind_points = num_ind_points,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 2)
        model_FSVA_eucl.set_optim_params(params={"init_cov_pars": result})
        model_FSVA_eucl.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fsva_eucl.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fsva_eucl = model_FSVA_eucl.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fsva_eucl.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fsva_eucl.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fsva_eucl['mu'])))
        matrix_CRPS_fsva_eucl.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fsva_eucl['mu'], pred_fsva_eucl['var'])
        matrix_LS_fsva_eucl.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fsva_eucl['mu'], pred_fsva_eucl['var'])

        pred_fsva_eucl_exp = model_FSVA_eucl.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fsva_eucl_exp['mu'])))
        matrix_CRPS_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fsva_eucl_exp['mu'], pred_fsva_eucl_exp['var'])
        matrix_LS_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fsva_eucl_exp['mu'], pred_fsva_eucl_exp['var'])

        
        
        start_time = time.time()
        # FSVA_corr
        model_FSVA_corr = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape = 2.5,num_neighbors = num_vecchia_neighbors,
                                 likelihood="gaussian",num_ind_points = num_ind_points,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 4)
        model_FSVA_corr.set_optim_params(params={"init_cov_pars": result})
        model_FSVA_corr.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fsva_corr.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fsva_corr = model_FSVA_corr.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fsva_corr.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fsva_corr.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fsva_corr['mu'])))
        matrix_CRPS_fsva_corr.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fsva_corr['mu'], pred_fsva_corr['var'])
        matrix_LS_fsva_corr.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fsva_corr['mu'], pred_fsva_corr['var'])

        pred_fsva_corr_exp = model_FSVA_corr.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fsva_corr.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fsva_corr_exp['mu'])))
        matrix_CRPS_exp_fsva_corr.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fsva_corr_exp['mu'], pred_fsva_corr_exp['var'])
        matrix_LS_exp_fsva_corr.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fsva_corr_exp['mu'], pred_fsva_corr_exp['var'])
        

tensor([[0.0877, 0.4387]])
0
torch.Size([10000, 2])
torch.Size([9916, 2])
torch.Size([10000, 2])
[GPBoost] [Warning] There is no intercept for modeling a possibly non-zero mean of the random effects. Consider including an intercept (= a column of 1's) in the covariates 'X' 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing points have been determined 
[GPBoost] [Warning] There is no intercept for modeling a possibly non-zero mean of the random effects. Consider including an intercept (= a column of 1's) in the covariates 'X' 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing points have been determined 
[GPBoost] [Warning] There is no intercept for modeling a possibly non-zero mean of the random effects. Consider including an intercept (= a column of 1's) in the covariates 'X' 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing

### Gaussian

In [21]:
# number of inducing points
num_ind_points = 200
# Different number of Vecchia neighbors
num_vecchia_neighbors = 30
# Number of input dimensions
vector_d = [2,10]
# number of reps
num_reps = 10
if toy:
    num_reps = 2
# Define row and column names
row_names = ['2-d','10-d']
col_names = ['Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6',
             'Iteration 7', 'Iteration 8', 'Iteration 9', 'Iteration 10']

# List of matrix names
matrix_names = [
    "matrix_t_vecchia", "matrix_t_fitc", "matrix_t_fsva_corr", "matrix_t_fsva_eucl",
    "matrix_tp_vecchia", "matrix_tp_fitc", "matrix_tp_fsva_corr", "matrix_tp_fsva_eucl",
    "matrix_RMSE_vecchia", "matrix_RMSE_fitc", "matrix_RMSE_fsva_corr", "matrix_RMSE_fsva_eucl",
    "matrix_LS_vecchia", "matrix_LS_fitc", "matrix_LS_fsva_corr", "matrix_LS_fsva_eucl",
    "matrix_CRPS_vecchia", "matrix_CRPS_fitc", "matrix_CRPS_fsva_corr", "matrix_CRPS_fsva_eucl",
    "matrix_RMSE_exp_vecchia", "matrix_RMSE_exp_fitc", "matrix_RMSE_exp_fsva_corr", "matrix_RMSE_exp_fsva_eucl",
    "matrix_LS_exp_vecchia", "matrix_LS_exp_fitc", "matrix_LS_exp_fsva_corr", "matrix_LS_exp_fsva_eucl",
    "matrix_CRPS_exp_vecchia", "matrix_CRPS_exp_fitc", "matrix_CRPS_exp_fsva_corr", "matrix_CRPS_exp_fsva_eucl"
]

# Create empty matrices and assign them dynamically to the global scope
for matrix_name in matrix_names:
    globals()[matrix_name] = pd.DataFrame(np.zeros((len(row_names), len(col_names))), 
                      index=row_names, columns=col_names)

# Nested loop to iterate over both vectors
for i, val1 in enumerate(vector_d):
    # Ranges
    ranges = torch.tensor([(0.15, 0.3,0.45,0.6,0.75)])
    avg_dist_5 = np.sqrt(5/6)
    if val1 != 5:
        # average distance
        avg_dist = np.sqrt(val1/6)
        min_ratio = avg_dist_5/0.15
        max_ratio = avg_dist_5/0.75
        min_range = avg_dist/min_ratio
        max_range = avg_dist/max_ratio
        ranges = torch.linspace(min_range, max_range, val1)
        ranges = ranges.unsqueeze(0)
        
    print(ranges)
    flattened = np.asarray(ranges).reshape(-1)
    result = np.concatenate(([0.3, 1], flattened))
    for j in range(num_reps):
        print(j)
        url = f"https://raw.githubusercontent.com/TimGyger/VIF/refs/heads/main/Simulation/Data/Gaussian/simulated_data_Gaussian_{val1}_{j}.txt"

        # Load the CSV file directly from the URL
        df = pd.read_csv(url)

        # Select the first d columns for X
        X = df.iloc[:, :val1]  # First d columns

        # Select the last column for y
        y = df.iloc[:, -1]  # Last column
        
        # Construct Test and Train sets 
        corner = 1 - 1/(4**(1/val1))
        X = torch.tensor(X.values, dtype=torch.float32)
        y = torch.tensor(y.values, dtype=torch.float32)
        upper_corner_mask = torch.all(X > corner, dim = 1)
        
        X_exp_test = X[upper_corner_mask]
        y_exp_test =  y[upper_corner_mask]
        X_remaining = X[~upper_corner_mask]
        y_remaining = y[~upper_corner_mask]
        X_int_test = X_remaining[:10000]
        y_int_test = y_remaining[:10000]
        X_train = X_remaining[10000:20000]
        y_train = y_remaining[10000:20000]
        
        print(X_train.size())
        print(X_exp_test.size())
        print(X_int_test.size())
        
        
        X_train = np.asarray(X_train)
        y_train = np.asarray(y_train)
        X_int_test = np.asarray(X_int_test)
        y_int_test = np.asarray(y_int_test)
        X_exp_test = np.asarray(X_exp_test)
        y_exp_test = np.asarray(y_exp_test)
        
        start_time = time.time()
        # Vecchia
        model_vecchia = gpb.GPModel(gp_coords=X_train, cov_function="gaussian_ard", 
                                 likelihood="gaussian",num_neighbors = num_vecchia_neighbors,
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 10)
        model_vecchia.set_optim_params(params={"init_cov_pars": result})
        model_vecchia.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_vecchia.loc[row_names[i], col_names[j]] = end_time - start_time

        start_time = time.time()
        pred_vecchia = model_vecchia.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_vecchia.loc[row_names[i], col_names[j]] = end_time - start_time
        matrix_RMSE_vecchia.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_vecchia['mu'])))
        matrix_CRPS_vecchia.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_vecchia['mu'], pred_vecchia['var'])
        matrix_LS_vecchia.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_vecchia['mu'], pred_vecchia['var'])

        pred_vecchia_exp = model_vecchia.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_vecchia.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_vecchia_exp['mu'])))
        matrix_CRPS_exp_vecchia.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_vecchia_exp['mu'], pred_vecchia_exp['var'])
        matrix_LS_exp_vecchia.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_vecchia_exp['mu'], pred_vecchia_exp['var'])

        start_time = time.time()
        # FITC
        model_FITC = gpb.GPModel(gp_coords=X_train, cov_function="gaussian_ard", 
                                 likelihood="gaussian",num_ind_points = 200,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="fitc",seed = 10)
        
        model_FITC.set_optim_params(params={"init_cov_pars": result})
        
        model_FITC.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fitc.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fitc = model_FITC.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fitc.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fitc.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fitc['mu'])))
        matrix_CRPS_fitc.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fitc['mu'], pred_fitc['var'])
        matrix_LS_fitc.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fitc['mu'], pred_fitc['var'])

        pred_fitc_exp = model_FITC.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fitc.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fitc_exp['mu'])))
        matrix_CRPS_exp_fitc.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fitc_exp['mu'], pred_fitc_exp['var'])
        matrix_LS_exp_fitc.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fitc_exp['mu'], pred_fitc_exp['var'])

        start_time = time.time()
        # FSVA_eucl
        model_FSVA_eucl = gpb.GPModel(gp_coords=X_train, cov_function="gaussian_ard", num_neighbors = num_vecchia_neighbors,
                                 likelihood="gaussian",num_ind_points = num_ind_points,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 2)
        model_FSVA_eucl.set_optim_params(params={"init_cov_pars": result})
        model_FSVA_eucl.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fsva_eucl.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fsva_eucl = model_FSVA_eucl.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fsva_eucl.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fsva_eucl.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fsva_eucl['mu'])))
        matrix_CRPS_fsva_eucl.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fsva_eucl['mu'], pred_fsva_eucl['var'])
        matrix_LS_fsva_eucl.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fsva_eucl['mu'], pred_fsva_eucl['var'])

        pred_fsva_eucl_exp = model_FSVA_eucl.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fsva_eucl_exp['mu'])))
        matrix_CRPS_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fsva_eucl_exp['mu'], pred_fsva_eucl_exp['var'])
        matrix_LS_exp_fsva_eucl.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fsva_eucl_exp['mu'], pred_fsva_eucl_exp['var'])

        
        
        start_time = time.time()
        # FSVA_corr
        model_FSVA_corr = gpb.GPModel(gp_coords=X_train, cov_function="gaussian_ard", num_neighbors = num_vecchia_neighbors,
                                 likelihood="gaussian",num_ind_points = num_ind_points,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 4)
        model_FSVA_corr.set_optim_params(params={"init_cov_pars": result})
        model_FSVA_corr.fit(y = y_train)
        # End the timer
        end_time = time.time()
        matrix_t_fsva_corr.loc[row_names[i], col_names[j]]  = end_time - start_time

        start_time = time.time()
        pred_fsva_corr = model_FSVA_corr.predict(gp_coords_pred = X_int_test, predict_var = True)
        end_time = time.time()
        matrix_tp_fsva_corr.loc[row_names[i], col_names[j]]  = end_time - start_time
        matrix_RMSE_fsva_corr.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_int_test - pred_fsva_corr['mu'])))
        matrix_CRPS_fsva_corr.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_int_test, pred_fsva_corr['mu'], pred_fsva_corr['var'])
        matrix_LS_fsva_corr.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_int_test, pred_fsva_corr['mu'], pred_fsva_corr['var'])

        pred_fsva_corr_exp = model_FSVA_corr.predict(gp_coords_pred = X_exp_test, predict_var = True)

        matrix_RMSE_exp_fsva_corr.loc[row_names[i], col_names[j]]  = np.sqrt(np.mean(np.square(y_exp_test - pred_fsva_corr_exp['mu'])))
        matrix_CRPS_exp_fsva_corr.loc[row_names[i], col_names[j]]  = crps_norm_vectorized(y_exp_test, pred_fsva_corr_exp['mu'], pred_fsva_corr_exp['var'])
        matrix_LS_exp_fsva_corr.loc[row_names[i], col_names[j]]  = log_score_norm_vectorized(y_exp_test, pred_fsva_corr_exp['mu'], pred_fsva_corr_exp['var'])
        

tensor([[0.0949, 0.4743]])
0
torch.Size([10000, 2])
torch.Size([9916, 2])
torch.Size([10000, 2])
[GPBoost] [Warning] There is no intercept for modeling a possibly non-zero mean of the random effects. Consider including an intercept (= a column of 1's) in the covariates 'X' 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing points have been determined 
[GPBoost] [Warning] There is no intercept for modeling a possibly non-zero mean of the random effects. Consider including an intercept (= a column of 1's) in the covariates 'X' 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing points have been determined 
[GPBoost] [Warning] There is no intercept for modeling a possibly non-zero mean of the random effects. Consider including an intercept (= a column of 1's) in the covariates 'X' 
[GPBoost] [Info] Starting kmeans++ algorithm for determining inducing points 
[GPBoost] [Info] Inducing